### Run basic `scanpy` QC and doublet detection with `scrublet` for **PBMC Tuberculosis** _Cai Y et al 2022_

**Objective**: Review QC process and suggest changes

- **Developed by**: Carlos Talavera-López PhD
- **Modified by**: Mairi McClean
- **Computational Health Centre - Helmholtz Munich**
- ORIGINAL: v221015; MODIFIED: v221116

### Load required modules

In [ ]:
import anndata
import logging
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sb
import scrublet as scr
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import rcParams

In [ ]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 160, color_map = 'RdPu', dpi_save = 180, vector_friendly = True, format = 'svg')

### Read datasets

In [ ]:
adata = sc.read_h5ad('/Users/mairi.mcclean/github/data/tb_pbmc_datasets/CaiY2022_TB.raw.h5ad')
adata

In [ ]:
adata.var

In [ ]:
adata.obs['donor'].value_counts()

In [ ]:
adata.obs['data_type'].values

In [ ]:
adata.var_names_make_unique()
sample_object = adata.copy()
sample_object

### Replace gene symbols 

In [ ]:
sample_object.var['gene_id'] = sample_object.var.index.copy()
sample_object.var.set_index('gene_name', inplace = True)
sample_object.var.head()

In [ ]:
sample_object.var_names = [str(i) for i in sample_object.var_names]
sample_object.var_names_make_unique()


### Inital scatterplot of top 20

In [ ]:

# highest fraction of counts per cell

sc.pl.highest_expr_genes(sample_object, n_top=20)

### Filter cells with less than 200 genes

In [ ]:
sc.pp.filter_cells(sample_object, min_genes = 200)
print(sample_object.n_obs, sample_object.n_vars)

In [ ]:
sample_object.shape

In [ ]:
sample_object.var

### Filter genes in less than 5 cells


In [ ]:
# Parameters (cells=3) taken from scanpy tutorial; others say 5? Will revisit
sc.pp.filter_genes(sample_object, min_cells=3)

filtered out 26143 genes that are detected in less than 3 cells


### Remove all data that is not scRNAseq

In [ ]:
sample_object.obs

,study,individual,sample,tissue,donor,data_type,centre,version,object,protocol,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_ribo,pct_counts_ribo,percent_mt2,n_counts
barcode,,,,,,,,,,,,,,,,,,,
AAACCTGGTATAGGGC-HRS180101-pbmc_scRNAseq,CaiY_2022,HRI111687,HRS180101,PBMC,P5,scRNAseq,Shenzhen University,10XV2,HRS180101,pbmc_scRNAseq,213,213,402.0,20.0,4.975124,6.0,1.492537,0.049751,402.0
AAACCTGTCTTTAGTC-HRS180101-pbmc_scRNAseq,CaiY_2022,HRI111687,HRS180101,PBMC,P5,scRNAseq,Shenzhen University,10XV2,HRS180101,pbmc_scRNAseq,206,206,421.0,53.0,12.589073,19.0,4.513064,0.125891,421.0
AAACGGGAGTTAACGA-HRS180101-pbmc_scRNAseq,CaiY_2022,HRI111687,HRS180101,PBMC,P5,scRNAseq,Shenzhen University,10XV2,HRS180101,pbmc_scRNAseq,201,201,312.0,28.0,8.974360,3.0,0.961538,0.089744,312.0
AAAGATGAGCAGACTG-HRS180101-pbmc_scRNAseq,CaiY_2022,HRI111687,HRS180101,PBMC,P5,scRNAseq,Shenzhen University,10XV2,HRS180101,pbmc_scRNAseq,237,237,430.0,33.0,7.674418,11.0,2.558140,0.076744,430.0
AAAGATGAGGACGAAA-HRS180101-pbmc_scRNAseq,CaiY_2022,HRI111687,HRS180101,PBMC,P5,scRNAseq,Shenzhen University,10XV2,HRS180101,pbmc_scRNAseq,255,255,464.0,52.0,11.206897,17.0,3.663793,0.112069,464.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAAGTGTTAGA-HRS100514-pfmc_scTCRseq,CaiY_2022,HRI068293,HRS100514,PFMC,P4,scTCRseq,Shenzhen University,10XV2,HRS100514,pfmc_scTCRseq,233,233,433.0,50.0,11.547344,14.0,3.233256,0.115473,433.0
TTTGTCACATCTACGA-HRS100514-pfmc_scTCRseq,CaiY_2022,HRI068293,HRS100514,PFMC,P4,scTCRseq,Shenzhen University,10XV2,HRS100514,pfmc_scTCRseq,264,264,441.0,44.0,9.977324,5.0,1.133787,0.099773,441.0
TTTGTCATCTAACTCT-HRS100514-pfmc_scTCRseq,CaiY_2022,HRI068293,HRS100514,PFMC,P4,scTCRseq,Shenzhen University,10XV2,HRS100514,pfmc_scTCRseq,271,271,572.0,127.0,22.202797,10.0,1.748252,0.222028,572.0


In [ ]:
sample_object.obs['data_type']

barcode
AAACCTGGTATAGGGC-HRS180101-pbmc_scRNAseq    scRNAseq
AAACCTGTCTTTAGTC-HRS180101-pbmc_scRNAseq    scRNAseq
AAACGGGAGTTAACGA-HRS180101-pbmc_scRNAseq    scRNAseq
AAAGATGAGCAGACTG-HRS180101-pbmc_scRNAseq    scRNAseq
AAAGATGAGGACGAAA-HRS180101-pbmc_scRNAseq    scRNAseq
                                              ...   
TTTGTCAAGTGTTAGA-HRS100514-pfmc_scTCRseq    scTCRseq
TTTGTCACATCTACGA-HRS100514-pfmc_scTCRseq    scTCRseq
TTTGTCATCTAACTCT-HRS100514-pfmc_scTCRseq    scTCRseq
TTTGTCATCTCGTTTA-HRS100514-pfmc_scTCRseq    scTCRseq
TTTGTCATCTTGTCAT-HRS100514-pfmc_scTCRseq    scTCRseq
Name: data_type, Length: 72235, dtype: category
Categories (2, object): ['scRNAseq', 'scTCRseq']

In [ ]:
sample_object.obs['data_type'].cat.categories

Index(['scRNAseq', 'scTCRseq'], dtype='object')

In [ ]:
# Code from https://scanpy.discourse.group/t/filter-out-specific-clusters-using-their-cluster-number/82

sample_object_new = sample_object[~sample_object.obs['data_type'].isin(['scTCRseq']),:]

In [ ]:
sample_object_new

View of AnnData object with n_obs × n_vars = 66560 × 35390
    obs: 'study', 'individual', 'sample', 'tissue', 'donor', 'data_type', 'centre', 'version', 'object', 'protocol', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_mt2', 'n_counts'
    var: 'gene_id', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'ribo'
    uns: 'donor_colors'

In [ ]:
sample_object_new.obs['data_type'].cat.categories

Index(['scRNAseq'], dtype='object')

### Compute QC stats

#### Remove mt genes

In [ ]:
sample_object_new.shape

In [ ]:
sample_object_new.var['mt'] = sample_object_new.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(sample_object_new, qc_vars = ['mt'], percent_top = None, log1p = False, inplace = True)
sample_object_new.var

#### Remove ribosomal RNA

###### When would you not remove ribosomal RNA?

In [ ]:
sample_object_new.var['ribo'] = sample_object_new.var_names.str.startswith(("RPS","RPL"))
sc.pp.calculate_qc_metrics(sample_object_new, qc_vars = ['ribo'], percent_top = None, log1p = False, inplace = True)
sample_object_new.var

,gene_id,n_cells,mt,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts,ribo
DDX11L1,ENSG00000223972.5,9,False,9,0.000138,99.987541,10.0,False
WASH7P,ENSG00000227232.5,41,False,41,0.000568,99.943241,41.0,False
OR4G11P,ENSG00000240361.2,4,False,4,0.000069,99.994463,5.0,False
OR4F5,ENSG00000186092.7,6,False,6,0.000083,99.991694,6.0,False
ENSG00000238009,ENSG00000238009.6,54,False,54,0.000761,99.925244,55.0,False
...,...,...,...,...,...,...,...,...
MT-ND6,ENSG00000198695.2,32012,True,32012,0.770444,55.683533,55653.0,False
MT-TE,ENSG00000210194.1,1062,True,1062,0.016031,98.529799,1158.0,False
MT-CYB,ENSG00000198727.2,43682,True,43682,2.059542,39.527930,148771.0,False
MT-TT,ENSG00000210195.2,84,True,84,0.001163,99.883713,84.0,False


In [ ]:
# for each cell compute fraction of counts in mito genes vs. all genes
# the `.A1` is only necessary as X is sparse (to transform to a dense array after summing)
# add the total counts per cell as observations-annotation to adata

mito_genes = sample_object_new.var_names.str.startswith('MT-')
sample_object_new.obs['percent_mt2'] = np.sum(
    sample_object_new[:, mito_genes].X, axis = 1).A1 / np.sum(sample_object_new.X, axis = 1).A1
sample_object_new.obs['n_counts'] = sample_object_new.X.sum(axis = 1).A1

In [ ]:
sample_object_new 

### Visualise QC metrics

In [ ]:
sample_object_new.var_names

In [ ]:
# From Anna's notebook?
sc.pl.violin(sample_object_new, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.violin(sample_object_new, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo'],
             jitter = 0.2, groupby = 'donor', rotation = 45)

In [ ]:
sc.pl.violin(sample_object_new, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo'],
             jitter=0.4, groupby = 'tissue', rotation = 45)

In [ ]:
sc.pl.scatter(sample_object_new, x = 'total_counts', y = 'pct_counts_mt', color = "donor")

In [ ]:
sc.pl.scatter(sample_object_new, x='total_counts', y='n_genes_by_counts', color = "donor")

### Filtering step

##### Values taken from Scanpy tutorial [https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html]

In [ ]:
sample_object_new = sample_object_new[sample_object_new.obs.n_genes_by_counts < 2500, :]
sample_object_new = sample_object_new[sample_object_new.obs.pct_counts_mt < 5, :]

### Add sample sex covariate

In [ ]:
annot = sc.queries.biomart_annotations(
        "hsapiens",
        ["ensembl_gene_id", "external_gene_name", "start_position", "end_position", "chromosome_name"],
    ).set_index("external_gene_name")

In [ ]:
annot.head()

In [ ]:
chrY_genes = sample_object_new.var_names.intersection(annot.index[annot.chromosome_name == "Y"])
chrY_genes

In [ ]:
sample_object_new.obs['percent_chrY'] = np.sum(
    sample_object_new[:, chrY_genes].X, axis = 1).A1 / np.sum(sample_object_new.X, axis = 1).A1 * 100

In [ ]:
sample_object_new.obs["XIST-counts"] = sample_object_new.X[:,sample_object_new.var_names.str.match('XIST')].toarray()

sc.pl.scatter(sample_object_new, x = 'XIST-counts', y = 'percent_chrY', color = "donor")

In [ ]:
sc.pl.violin(sample_object_new, ["XIST-counts", "percent_chrY"], jitter = 0.4, groupby = 'donor', rotation = 45)

### Calculate cell cycle scores

In [ ]:
!if [ ! -f /Users/mairi.mcclean/cell_cycle_gene.txt ]; then curl -o /Users/mairi.mcclean/cell_cycle_gene.txt https://raw.githubusercontent.com/theislab/scanpy_usage/master/180209_cell_cycle/data/regev_lab_cell_cycle_genes.txt; fi

In [ ]:
cell_cycle_genes = [x.strip() for x in open('/Users/mairi.mcclean/cell_cycle_gene.txt')]
print(len(cell_cycle_genes))

# Split into 2 lists
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]

cell_cycle_genes = [x for x in cell_cycle_genes if x in sample_object_new.var_names]
print(len(cell_cycle_genes))

- Create basic `anndata` for score calculation

In [ ]:
adata_log = anndata.AnnData(X = sample_object_new.X,  var = sample_object_new.var, obs = sample_object_new.obs)
sc.pp.normalize_total(adata_log, target_sum = 1e6, exclude_highly_expressed = True)
sc.pp.log1p(adata_log)

In [ ]:
sc.tl.score_genes_cell_cycle(adata_log, s_genes = s_genes, g2m_genes = g2m_genes)
sc.pl.violin(adata_log, ['S_score', 'G2M_score'],
             jitter = 0.4, groupby = 'donor', rotation = 45)

In [ ]:
sample_object_new.obs['S_score'] = adata_log.obs['S_score']
sample_object_new.obs['G2M_score'] = adata_log.obs['G2M_score']
sample_object_new

### Predict doublets

In [ ]:
scrub = scr.Scrublet(sample_object_new.X)
sample_object_new.obs['doublet_scores'], sample_object_new.obs['predicted_doublets'] = scrub.scrub_doublets()
scrub.plot_histogram()

sum(sample_object_new.obs['predicted_doublets'])

In [ ]:
#check if our predicted doublets also have more detected genes in general

sc.pl.violin(sample_object_new, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'predicted_doublets'],
             jitter = 0.2, groupby = 'donor', rotation = 45)

In [ ]:
sc.pl.violin(sample_object_new, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'predicted_doublets'],
             jitter = 0.2, groupby = 'sample', rotation = 45)

### Prepare counts for individual slots

#### Preparation for data normalization
>> not QC?

In [ ]:
sample_object_new.raw = sample_object_new.copy()
sample_object_new.layers['counts'] = sample_object_new.X.copy()
sample_object_new.layers["sqrt_norm"] = np.sqrt(
    sc.pp.normalize_total(sample_object_new, inplace = False)["X"]
)
sample_object_new

### Identification of highly variable genes

#### Where should this be done? After log transformation?

In [ ]:
sc.pp.highly_variable_genes(sample_object_new, min_mean=0.0125, max_mean=3, min_disp=0.5)


In [ ]:
sc.pl.highly_variable_genes(sample_object_new)


### Export object

In [ ]:
sample_object_new.write('/home/cartalop/data/single_cell/lung/tb/caiy2022/CaiY2022_TB_QCed_pre-process_ctl221015.h5ad')